In [1]:
# 파일 경로 주석
# X.csv, y.csv 파일을 사용합니다.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 데이터 불러오기
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
y = y.iloc[:, 0]

# 2. 데이터 누수(Data Leakage) 원인인 'cust_id' 컬럼 제거
X = X.drop('cust_id', axis=1)

# 3. 데이터 전처리
# (1) 문자열 데이터를 숫자형으로 변환 (원-핫 인코딩)
X_dummies = pd.get_dummies(X, drop_first=True)
# (2) 결측치를 각 컬럼의 평균값으로 대체
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_dummies)
# (3) 다시 DataFrame으로 변환
X_processed = pd.DataFrame(X_imputed, columns=X_dummies.columns)

# 4. 교차 검증(CV)을 하지 않는 모델 성능 평가
# 데이터를 훈련용과 테스트용으로 8:2로 분할
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# 선형 회귀 (단일 모델)
lr_single = LinearRegression()
lr_single.fit(X_train, y_train)
lr_single_preds = lr_single.predict(X_test)
lr_single_rmse = np.sqrt(mean_squared_error(y_test, lr_single_preds))
lr_single_r2 = r2_score(y_test, lr_single_preds)

# 랜덤 포레스트 (단일 모델)
rf_single = RandomForestRegressor(n_estimators=100, random_state=42)
rf_single.fit(X_train, y_train)
rf_single_preds = rf_single.predict(X_test)
rf_single_rmse = np.sqrt(mean_squared_error(y_test, rf_single_preds))
rf_single_r2 = r2_score(y_test, rf_single_preds)

# 5. 교차 검증(CV)을 이용한 모델 성능 평가
# 선형 회귀 (CV)
lr_cv = LinearRegression()
lr_cv_rmse_scores = np.sqrt(-cross_val_score(lr_cv, X_processed, y, cv=5, scoring='neg_mean_squared_error'))
lr_cv_r2_scores = cross_val_score(lr_cv, X_processed, y, cv=5, scoring='r2')
lr_cv_rmse = lr_cv_rmse_scores.mean()
lr_cv_r2 = lr_cv_r2_scores.mean()

# 랜덤 포레스트 (CV)
rf_cv = RandomForestRegressor(n_estimators=100, random_state=42)
rf_cv_rmse_scores = np.sqrt(-cross_val_score(rf_cv, X_processed, y, cv=5, scoring='neg_mean_squared_error'))
rf_cv_r2_scores = cross_val_score(rf_cv, X_processed, y, cv=5, scoring='r2')
rf_cv_rmse = rf_cv_rmse_scores.mean()
rf_cv_r2 = rf_cv_r2_scores.mean()

# 6. 결과 정리 및 시각화
# 결과를 DataFrame으로 정리
results_data = {
    'Model': ['Linear Regression', 'Linear Regression', 'Random Forest', 'Random Forest'],
    'Validation': ['No CV', 'CV', 'No CV', 'CV'],
    'RMSE': [lr_single_rmse, lr_cv_rmse, rf_single_rmse, rf_single_rmse],
    'R2 Score': [lr_single_r2, lr_cv_r2, rf_single_r2, rf_cv_r2]
}
results_df = pd.DataFrame(results_data)

# 차트 그리기
plt.rc('font', family='Malgun Gothic') # 한글 폰트 설정
plt.style.use('seaborn-v0_8-whitegrid')
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# RMSE Plot
sns.barplot(x='Model', y='RMSE', hue='Validation', data=results_df, ax=axes[0], palette='viridis')
axes[0].set_title('모델 비교: RMSE (낮을수록 좋음)', fontsize=16)
axes[0].set_xlabel('모델', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
for p in axes[0].patches:
    axes[0].annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                     ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                     textcoords='offset points')

# R2 Score Plot
sns.barplot(x='Model', y='R2 Score', hue='Validation', data=results_df, ax=axes[1], palette='plasma')
axes[1].set_title('모델 비교: R² Score (1에 가까울수록 좋음)', fontsize=16)
axes[1].set_xlabel('모델', fontsize=12)
axes[1].set_ylabel('R² Score', fontsize=12)
axes[1].set_ylim(0, 1) # R2 Score의 y축 범위를 0과 1 사이로 고정
for p in axes[1].patches:
    axes[1].annotate(f'{p.get_height():.4f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                     ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                     textcoords='offset points')

plt.tight_layout()
plt.savefig('realistic_model_comparison_charts.png')

# 최종 결과를 표로 출력
print("--- 최종 모델 성능 비교 결과 ('cust_id' 제거 후) ---")
print(results_df)

FileNotFoundError: [Errno 2] No such file or directory: 'X.csv'